# <center>Predictive modelling with timeseries</center>
# <center> Part 3 - Time series forecasting with ARIMA models</center>

![Image](images/timeseries.jpg)

# Time series forecasting with ARIMA models

Very robust family of parametric models that can handle time series data with many combinations of properties.
ARIMA models have three components, the `AR` autoregressive, the `I` integrated, and `MA` moving average. the notation is put together as `ARIMA(p, d, q)`.


![Image](images/arima.png) 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# jupyter lab configs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)

from utils import adf_test
from utils import print_errors

In [ ]:
from statsmodels.tsa.arima_model import ARIMA,ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pmdarima import auto_arima # on order to determine ARIMA orders
from statsmodels.tsa.stattools import adfuller

In [ ]:
import pmdarima
pmdarima.__version__
plotly.__version__

# ETL

### Load the datasets  (Production of electrical equipments)

In [ ]:
# load a nice example for decomposition - production of electrical equipments
ele_df = pd.read_csv('datasets/elecequip.csv')
ele_df['value'].plot()

---

### Split train and test

In [ ]:
train = ele_df.loc[:150, 'value']
test = ele_df.loc[150:, 'value']
test_start = 150
test_end = 194

## Create baseline and check model performance

---

# Auto-regressive family of models

### Is the series  `stationary`?

In [ ]:
adf_test(train)

# ARMA - For a stationary series

What happens if we model this as a stationary series?
For ARMA and stationary data, we don't need the Integration/differencing parameter `d`, which is then set to 0.  We are then left with a ARMA model and heve to find the best `p` and `q`.
Let's see the impact of changing `p` and  `q`.

In [ ]:
arma_model = ARIMA(train, order=(1,1,1))
results1_1 = arma_model.fit()
results1_1.summary().tables[0]

arma_model = ARIMA(train, order=(2,0,2))
results2_2 = arma_model.fit()
results2_2.summary().tables[0]

arma_model = ARIMA(train, order=(1,0,2))
results1_2 = arma_model.fit()
results1_2.summary().tables[0]

arma_model = ARIMA(train, order=(5,1,1))
results2_1 = arma_model.fit()
results2_1.summary().tables[0]

arma_model2 = ARIMA(train, order=(1,1,1))
results2_1 = arma_model2.fit()
results2_1.summary().tables[0]

Let's predict for the test interval:

In [ ]:
predictions1 = results1_1.predict(start=test_start, end=test_end).rename('ARMA(1,1) Predictions')
predictions2 = results2_2.predict(start=test_start, end=test_end).rename('ARMA(2,2) Predictions')
predictions3 = results1_2.predict(start=test_start, end=test_end).rename('ARMA(1,2) Predictions')
predictions4 = results2_1.predict(start=test_start, end=test_end).rename('ARMA(5,1) Predictions')

In [ ]:
ylabel='Demand'
ax = train.plot(legend=True,figsize=(12,6),title='Prediction of equiment demand')
test.plot(legend=True)
predictions1.plot(legend=True)
predictions2.plot(legend=True)
predictions3.plot(legend=True)
predictions4.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel="", ylabel=ylabel)

Did you notice the change in shape of predictions when we increase the AR component to 5?

# Wait a minute!  
## Do we have to try out and plot all possible combinations of (p, d, q)?
![Image](images/boring.jpg) 

## Luckly not.   

### Using AIC and a stewise routine `auto_arima()` made available in library `pmdarima`, we can find the best (p, d, q) for the data

💡 **Check also:** you can check this very nice implementation of a **grid search** of ARIMA models:
[https://machinelearningmastery.com/how-to-grid-search-sarima-model-hyperparameters-for-time-series-forecasting-in-python/]

# Stepwise ARIMA or SARIMA with `auto_arima()`

The function `auto_arima()` will fit different combinations of the orders of (p, d, q) and select the best based on the **lowest AIC**. You can indicate a seasonal model (SARIMA) if you set `seasonal=True` and change `m` to integer>1.

In [ ]:
best_model = auto_arima(train, suppress_warnings=True, 
                        random=False, start_p=0, start_q=0, 
                        stepwise=False,
                        random_state=42)
best_model.summary()

### What does it mean? 

The `auto_arima` is telling us that the best model (= lowest AIC) to treat these data is a 
> ARIMA(5, 1, 2) 

*p.s. An SARIMAX or SARIMA without a (P,D,Q)m component is the equivalent of an ARIMA*

### Now, what is the performance on actual test data?

In [ ]:
best_model = ARIMA(train, order=(2,1,3))
results = best_model.fit()

# predict
predictions = results.predict(start=test_start, end=test_end, 
                              dynamic=False, typ='levels').rename('SARIMAX(5,1,2) Predictions')

## How is the forecasting looking like in the `test` data?

In [ ]:
ylabel='Demand'
ax = test.plot(legend=True,figsize=(12,6),title='Prediction of equiment demand')
test.plot()
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel="", ylabel=ylabel)

In [ ]:
fig = px.line(test, title="Equipment demand - test data and predictions",  width=900, height=500)
fig.add_trace(go.Line(x=test.index, y=predictions,
                    mode='lines', name='SARIMAX'))
fig.show()

## Model evaluation:

In [ ]:
print_errors(test, predictions, 'Test set')

----

### Can we improve our predictions?
# Adding the seasonal component (SARIMA)

In [ ]:
best_model = auto_arima(train, suppress_warnings=True, start_p=0, start_q=0, m=6)
best_model.summary()

In [ ]:
best_model = SARIMAX(train, order=(3,1,0), seasonal_order=(2,0,1,6))
results = best_model.fit()

# predict
predictions = results.predict(start=test_start, end=test_end, 
                              dynamic=False, typ='levels').rename('SARIMAX Predictions')
print_errors(test, predictions, 'Test set')

In [ ]:
fig = px.line(test, title="Equipment demand - test data and predictions",  width=900, height=500)
fig.add_trace(go.Line(x=test.index, y=predictions,
                    mode='lines', name='SARIMAX'))
fig.show()

---

# Working with regressors - SARIMA**x**

In [ ]:
# training data
rossman_df = pd.read_csv('datasets/rossman_train.csv').reset_index(drop=True)
# set the index to the time column
rossman_df.Date = pd.to_datetime(rossman_df.Date)

# load store info
stores = pd.read_csv('datasets/rossman_store.csv').reset_index(drop=True)

# merge store and sales
rossman_df = pd.merge(rossman_df, stores, how='left', on='Store')
rossman_df = rossman_df[rossman_df.Store==1]

rossman_df.set_index('Date', drop=True, inplace=True)
rossman_df.head(4)

## Basic demonstration of the library usage
We need to add several polishing steps

In [ ]:
# Fit a model without any extra regressors
mod = SARIMAX(rossman_df.Sales, order=(1,1,0), seasonal_order=(2,0,1,7))
res = mod.fit(disp=False)
print(res.summary())

In [ ]:
# add timeseries (endog) and regressors (exog)
endog = rossman_df.Sales
exog = rossman_df.Customers

# Fit the model
mod2 = SARIMAX(endog, exog, order=(1,1,0), seasonal_order=(2,0,1,7))
res = mod2.fit(disp=False)
print(res.summary())

---

<a href='https://www.freepik.com/vectors/business'>Business vector created by freepik - www.freepik.com</a>